In [ ]:
using Distributed
addprocs(2)
@everywhere using Scythe
@everywhere using DistributedData
@everywhere using SharedArrays
@everywhere using SparseArrays
using CSV
using DataFrames
using PyPlot
using Glob
using DSP, LinearRegression
using Statistics
using FileIO, ImageIO, Colors, FixedPointNumbers, Images
using PyCall

In [ ]:
spinup = "./symmetric_spinup"
if !isdir(spinup)
    mkdir(spinup)
end

In [ ]:
model = ModelParameters(
    ts = 3.0,
    integration_time = 10800.0,
    output_interval = 3600.0,
    equation_set = "Oneway_ShallowWater_Slab",
    initial_conditions = "./symmetric_spinup/SWslab_OnewayRankine.csv",
    output_dir = "./symmetric_spinup/",
    grid_params = GridParameters(
        geometry="RL",
        xmin = 0.0,
        xmax = 3.0e5,
        num_cells = 100,
        BCL = Dict(
            "h" => CubicBSpline.R1T1,
            "u" => CubicBSpline.R1T0,
            "v" => CubicBSpline.R1T0,
            "ub" => CubicBSpline.R1T0,
            "vb" => CubicBSpline.R1T0,
            "wb" => CubicBSpline.R1T1),
        BCR = Dict(
            "h" => CubicBSpline.R0,
            "u" => CubicBSpline.R1T1,
            "v" => CubicBSpline.R0,
            "ub" => CubicBSpline.R1T1,
            "vb" => CubicBSpline.R0,
            "wb" => CubicBSpline.R0),
        vars = Dict(
            "h" => 1,
            "u" => 2,
            "v" => 3,
            "ub" => 4,
            "vb" => 5,
            "wb" => 6)),
        physical_params = Dict(
            :g => 9.81,
            :K => 3000.0,
            :Cd => 2.4e-3,
            :Hfree => 2000.0,
            :Hb => 1000.0,
            :f => 5.0e-5))

In [ ]:
scythegrid = createGrid(model.grid_params)
gridpoints = Scythe.getGridpoints(scythegrid)
cartesian = Scythe.getCartesianGridpoints(scythegrid);

In [ ]:
Rmax = 50000.0
Vmax = 50.0
V0 = Vmax / Rmax
zeta = 2.0 * V0
epsilon = 5000.0
f = 5.0e-5
r_1 = 0.0
h = 0.0
ubar = 0.0
vbar = 0.0
vprime = 0.0
uprime = 0.0
for i = 1:grid.params.lDim
    r_m = gridpoints[i,1]
    l_m = gridpoints[i,2]
    x_m = cartesian[i,1]
    y_m = cartesian[i,2]
    if (r_m < Rmax)
        vbar = V0 * r_m
        #vprime = 0.5 * zeta * r_m * (epsilon * cos(2 * l_m) / Rmax)
        #uprime = 0.5 * zeta * r_m * (epsilon * sin(2 * l_m) / Rmax)
    else
        vbar = Rmax * Rmax * V0 / r_m
        #vprime = 0.5 * zeta * (Rmax^2 / r_m) * (-epsilon * cos(2 * l_m) * Rmax / r_m^2)
        #uprime = 0.5 * zeta * (Rmax^2 / r_m) * (epsilon * sin(2 * l_m) * Rmax / r_m^2)
    end
    if r_m > r_1
        dhdr = ((f * vbar) + (vbar * vbar / r_m)) / 9.81
        h = h + (dhdr * (r_m - r_1))
        r_1 = r_m
    end
    grid.physical[i,1,1] = h
    grid.physical[i,2,1] = ubar
    grid.physical[i,3,1] = vbar
    grid.physical[i,4,1] = ubar
    grid.physical[i,5,1] = vbar
    grid.physical[i,6,1] = 0.0
end

In [ ]:
rankine_ics = DataFrame(r=gridpoints[:,1],
    l=gridpoints[:,2], 
    h=grid.physical[:,1,1], 
    u=grid.physical[:,2,1], 
    v=grid.physical[:,3,1],
    ub = grid.physical[:,4,1],
    vb = grid.physical[:,5,1],
    wb = grid.physical[:,6,1]); 

In [ ]:
CSV.write(spinup*"/SWslab_OnewayRankine.csv",rankine_ics)

In [ ]:
# Run the 3-hr spinup
integrate_model(model)

In [ ]:
@pyimport numpy as np
@pyimport scipy.interpolate as interpolation
@pyimport scipy.stats as stats
@pyimport scipy.signal as sig
@pyimport matplotlib.pyplot as cm
@pyimport matplotlib.patches as patches
@pyimport scipy as sp
@pyimport scipy.interpolate as sp_interp
@pyimport statsmodels.api as sm
@pyimport statsmodels.tsa.stattools as stattools #import grangercausalitytests
# >>> import numpy as np

In [ ]:
include("scythe_func.jl")

In [ ]:
xfont = 20
fig_text_default = xfont
fig_title_default = fig_text_default*1.2
linewidth_default = 5
plt.rc("lines", linewidth=linewidth_default)    
plt.rc("text", usetex=false)
plt.rc("font", size=fig_text_default, weight="normal",family="sans-serif")
plt.rc("axes",titlesize=fig_title_default,titleweight="normal")
plt.rc("figure", dpi = 300)
color_of_data = ["lightslategray", "dodgerblue", "mediumaquamarine", "coral"];

In [ ]:
output = "./symmetric_spinup/"
cd(output)
file_list=glob("gridded_out*")
plots = "plots"
if !isdir(plots)
    mkdir(plots)
end
if !isdir("gif")
    mkdir("gif")
end
nodes = 100;
azi = 601;
numCoeffs = 5;
interval= 120;

In [ ]:
h_lls = zeros(Float64,nodes,azi,Int(round.(numCoeffs/2))+1,length(file_list))
u_lls = zeros(Float64,nodes,azi,Int(round.(numCoeffs/2))+1,length(file_list))
v_lls = zeros(Float64,nodes,azi,Int(round.(numCoeffs/2))+1,length(file_list));
ub_lls = zeros(Float64,nodes,azi,Int(round.(numCoeffs/2))+1,length(file_list))
vb_lls = zeros(Float64,nodes,azi,Int(round.(numCoeffs/2))+1,length(file_list));
wb_lls = zeros(Float64,nodes,azi,Int(round.(numCoeffs/2))+1,length(file_list))

vort_lls = zeros(Float64,nodes,azi,Int(round.(numCoeffs/2))+1,length(file_list))
div_lls = zeros(Float64,nodes,azi,Int(round.(numCoeffs/2))+1,length(file_list));
vortb_lls = zeros(Float64,nodes,azi,Int(round.(numCoeffs/2))+1,length(file_list))
divb_lls = zeros(Float64,nodes,azi,Int(round.(numCoeffs/2))+1,length(file_list));
pv_lls = zeros(Float64,nodes,azi,Int(round.(numCoeffs/2))+1,length(file_list))

h_amp = zeros(Float64,nodes,Int(round.(numCoeffs/2)),length(file_list))
u_amp = zeros(Float64,nodes,Int(round.(numCoeffs/2)),length(file_list))
v_amp = zeros(Float64,nodes,Int(round.(numCoeffs/2)),length(file_list));
ub_amp = zeros(Float64,nodes,Int(round.(numCoeffs/2)),length(file_list))
vb_amp = zeros(Float64,nodes,Int(round.(numCoeffs/2)),length(file_list))
wb_amp = zeros(Float64,nodes,Int(round.(numCoeffs/2)),length(file_list));

vort_amp = zeros(Float64,nodes,Int(round.(numCoeffs/2)),length(file_list))
div_amp = zeros(Float64,nodes,Int(round.(numCoeffs/2)),length(file_list));
vortb_amp = zeros(Float64,nodes,Int(round.(numCoeffs/2)),length(file_list))
divb_amp = zeros(Float64,nodes,Int(round.(numCoeffs/2)),length(file_list));
pv_amp = zeros(Float64,nodes,Int(round.(numCoeffs/2)),length(file_list));

In [ ]:
output = CSV.read(file_list[1], DataFrame, header=1)
radius = reshape(output.r,azi,nodes)[1,:]
lambda = reshape(output.l,azi,nodes)[:,1];
f = 1e-5;
vmax = []
RMW = []
ind_RMW = []
meanH = 2000;

In [ ]:
num_out = length(file_list)
for i in 1:num_out
    output = CSV.read(file_list[i], DataFrame, header=1)
#     println(file_list[i])
    radius = reshape(output.r,azi,nodes)[1,:]
    lambda = reshape(output.l,azi,nodes)[:,1]
    h = reshape(output.h,azi,nodes)'
    hr = reshape(output.h_r,azi,nodes)'
    u = reshape(output.u,azi,nodes)'
    v = reshape(output.v,azi,nodes)'
    x = reshape(output.x,azi,nodes)'
    y = reshape(output.y,azi,nodes)';
    ub = reshape(output.ub,azi,nodes)'
    vb = reshape(output.vb,azi,nodes)';
    wb = reshape(output.wb,azi,nodes)';
    v_dr = reshape(output.v_r,azi,nodes)'
    u_dl = reshape(output.u_l,azi,nodes)'
    vb_dr = reshape(output.v_r,azi,nodes)'
    ub_dl = reshape(output.u_l,azi,nodes)'
    u_dr = reshape(output.u_r,azi,nodes)'
    v_dl = reshape(output.v_l,azi,nodes)'
    ub_dr = reshape(output.ub_r,azi,nodes)'
    vb_dl = reshape(output.vb_l,azi,nodes)'
    
    vort = v_dr .+ (v./radius) .+ f .- (u_dl./radius)
    div = u_dr .+ (u./radius) .- (v_dl./radius)
    vortb = vb_dr .+ (vb./radius) .+ f .- (ub_dl./radius)
    divb = ub_dr .+ (ub./radius) .- (vb_dl./radius)
    pv = (vort .+ f) ./ (h .+ meanH);

    append!(vmax, findmax(v)[1])
    append!(RMW, radius[findmax(v)[2][1]])
    append!(ind_RMW, findmax(v)[2][1])

    h_lls[:,:,:,i], h_amp[:,:,i] = wave_reconstruc(radial_lls(h, lambda, numCoeffs), lambda)
    u_lls[:,:,:,i], u_amp[:,:,i] = wave_reconstruc(radial_lls(u, lambda, numCoeffs), lambda)
    v_lls[:,:,:,i], v_amp[:,:,i] = wave_reconstruc(radial_lls(v, lambda, numCoeffs), lambda)
    ub_lls[:,:,:,i], ub_amp[:,:,i] = wave_reconstruc(radial_lls(ub, lambda, numCoeffs), lambda)
    vb_lls[:,:,:,i], vb_amp[:,:,i] = wave_reconstruc(radial_lls(vb, lambda, numCoeffs), lambda)
    wb_lls[:,:,:,i], wb_amp[:,:,i] = wave_reconstruc(radial_lls(wb, lambda, numCoeffs), lambda)

    vort_lls[:,:,:,i], vort_amp[:,:,i] = wave_reconstruc(radial_lls(vort, lambda, numCoeffs), lambda)
    div_lls[:,:,:,i], div_amp[:,:,i] = wave_reconstruc(radial_lls(div, lambda, numCoeffs), lambda)
    vortb_lls[:,:,:,i], vortb_amp[:,:,i] = wave_reconstruc(radial_lls(vortb, lambda, numCoeffs), lambda)
    divb_lls[:,:,:,i], divb_amp[:,:,i] = wave_reconstruc(radial_lls(divb, lambda, numCoeffs), lambda)
    pv_lls[:,:,:,i], pv_amp[:,:,i] = wave_reconstruc(radial_lls(pv, lambda, numCoeffs), lambda)
end

In [ ]:
vort = zeros(length(radius),length(file_list));
dvortdr = zeros(length(radius),length(file_list));
PV = zeros(length(radius),length(file_list));
for i in eachindex(file_list)
    vort[:,i] = vort_lls[:,1,1,i]
    dvortdr[:,i] = finite_dr(radius,vort[:,i]);
    PV[:,i] = (vort[:,i] .+ f) ./ (h_lls[:,1,1,i] .+ 2000)
end

In [ ]:
plt.figure(figsize=(12,12))
plt.subplots_adjust(hspace=0.1)
plt.subplot(2,1,1)
ax1 = gca()
plt.title("(a)") #Time = "*string(time[4]*3600)*" "*string("s"))
plt.plot(radius ./ 1000, v_lls[:,1,1,num_out], color="k", label=L"WN0 V$_T$")
plt.plot(radius ./ 1000, vb_lls[:,1,1,num_out], color=color_of_data[1], label=L"WN0 V$_b$")
plt.legend(loc=2)
ax1.set_xlim(0,80)
ax1.set_ylim(0,70)
ax1.set_ylabel(L"m s$^{-1}$")
ax1.tick_params(axis="x", labelcolor="w", size=8)
PyPlot.grid("on")
ax2 = ax1.twinx() 
ax2.plot(radius ./ 1000, ub_lls[:,1,1,num_out], color=color_of_data[3], label=L"WN0 U$_b$")
ax2.plot(radius ./ 1000, wb_lls[:,1,1,num_out], color=color_of_data[4], label=L"WN0 W$_b$")
ax2.set_ylim(-25,10)
ax2.set_ylabel(L"m s$^{-1}$")

plt.xticks(color="w", fontsize=8)
# ax2.set_xticks(color="w", fontsize=8)
plt.legend(loc=1)
plt.subplot(2,1,2)
ax1 = gca()
plt.title("(b)")
plt.plot(radius ./ 1000, vort[:,num_out], color="k", label=L"WN0 $\zeta$")
plt.legend(loc="upper right", bbox_to_anchor=(0.25, 0.85))
plt.xlim(0,80)
ax1.set_ylabel(L"s$^{-1}$")
PyPlot.grid("on")
plt.ylim(-1.2e-4,2.2e-3)
plt.xlabel("Radius (km)")

ax2 = ax1.twinx() 
ax2.plot(radius ./ 1000, h_lls[:,1,1,num_out], color=color_of_data[4], label="WN0 h")
plt.legend(loc="upper right", bbox_to_anchor=(1, 0.85))
ax2.set_ylabel("m")
ax2.set_ylim(-10,220)
plt.savefig(plots*"/Cha_Bell_Fig2.pdf", transparent=true, bbox_inches="tight")

In [ ]:
balanced = CSV.read("./symmetric_spinup/physical_out_10800.0.csv", DataFrame, header=1);

In [ ]:
Rmax = 50000.0
Vmax = 50.0
V0 = Vmax / Rmax
zeta = 2.0 * V0
epsilon = 5000.0
f = 5.0e-5
r_1 = 0.0
h = 0.0
ubar = 0.0
vbar = 0.0
vprime = 0.0
uprime = 0.0
for i = 1:scythegrid.params.lDim
    r_m = gridpoints[i,1]
    l_m = gridpoints[i,2]
    x_m = cartesian[i,1]
    y_m = cartesian[i,2]
    R_eli = Rmax + (epsilon * cos(2 * l_m))
    
    # Use the output field for mean vortex
    vfree = balanced.v[i]
    ufree = balanced.u[i]
    h = balanced.h[i]
    vbl = balanced.vb[i]
    ubl = balanced.ub[i]
    wbl = balanced.wb[i]    

    if (r_m < Rmax)
        vprime = 0.5 * zeta * r_m * (epsilon * cos(2 * l_m) / Rmax)
        uprime = 0.5 * zeta * r_m * (epsilon * sin(2 * l_m) / Rmax)
    else
        vprime = 0.5 * zeta * (Rmax^2 / r_m) * (-epsilon * cos(2 * l_m) * Rmax / r_m^2)
        uprime = 0.5 * zeta * (Rmax^2 / r_m) * (epsilon * sin(2 * l_m) * Rmax / r_m^2)
    end
    scythegrid.physical[i,1,1] = h
    scythegrid.physical[i,2,1] = ufree + uprime
    scythegrid.physical[i,3,1] = vfree + vprime
    scythegrid.physical[i,4,1] = ubl + uprime
    scythegrid.physical[i,5,1] = vbl + vprime
    scythegrid.physical[i,6,1] = wbl
end

In [ ]:
wave2_ics = DataFrame(r=gridpoints[:,1],
    l=gridpoints[:,2], 
    h=scythegrid.physical[:,1,1], 
    u=scythegrid.physical[:,2,1], 
    v=scythegrid.physical[:,3,1],
    ub = scythegrid.physical[:,4,1],
    vb = scythegrid.physical[:,5,1],
    wb = scythegrid.physical[:,6,1]);

In [ ]:
oneway = "./oneway_wave2"
if !isdir(oneway)
    mkdir(oneway)
end

In [ ]:
CSV.write(oneway*"/SWslab_wave2.csv",wave2_ics)